In [4]:
#Here is a comment for installing chembl_webresource_client
pip install chembl_webresource_client

Note: you may need to restart the kernel to use updated packages.


In [3]:
#importing necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from chembl_webresource_client.new_client import new_client
from rdkit import Chem
from rdkit.Chem import Descriptors,Lipinski,rdchem

In [4]:
#searching and selecting  as the drug target
target_query = new_client.target.search("Erbb1")
targets = pd.DataFrame(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,"[{'xref_id': 'P00533', 'xref_name': None, 'xre...",Homo sapiens,Epidermal growth factor receptor erbB1,15.0,False,CHEMBL203,"[{'accession': 'P00533', 'component_descriptio...",SINGLE PROTEIN,9606
1,[],Homo sapiens,Protein cereblon/Epidermal growth factor receptor,14.0,False,CHEMBL4523680,"[{'accession': 'P00533', 'component_descriptio...",PROTEIN-PROTEIN INTERACTION,9606
2,[],Homo sapiens,MER intracellular domain/EGFR extracellular do...,13.0,False,CHEMBL3137284,"[{'accession': 'P00533', 'component_descriptio...",CHIMERIC PROTEIN,9606
3,[],Homo sapiens,EGFR/PPP1CA,13.0,False,CHEMBL4523747,"[{'accession': 'P00533', 'component_descriptio...",PROTEIN-PROTEIN INTERACTION,9606
4,[],Homo sapiens,VHL/EGFR,13.0,False,CHEMBL4523998,"[{'accession': 'P00533', 'component_descriptio...",PROTEIN-PROTEIN INTERACTION,9606
5,"[{'xref_id': 'Q01279', 'xref_name': None, 'xre...",Mus musculus,Epidermal growth factor receptor erbB1,12.0,False,CHEMBL3608,"[{'accession': 'Q01279', 'component_descriptio...",SINGLE PROTEIN,10090
6,[],Homo sapiens,Epidermal growth factor receptor and ErbB2 (HE...,10.0,False,CHEMBL2111431,"[{'accession': 'P04626', 'component_descriptio...",PROTEIN FAMILY,9606
7,[],Homo sapiens,Baculoviral IAP repeat-containing protein 2/Ep...,10.0,False,CHEMBL4802031,"[{'accession': 'P00533', 'component_descriptio...",PROTEIN-PROTEIN INTERACTION,9606
8,[],Homo sapiens,Epidermal growth factor receptor,7.0,False,CHEMBL2363049,"[{'accession': 'P04626', 'component_descriptio...",PROTEIN FAMILY,9606


In [5]:
selected_query = targets.target_chembl_id[0]
selected_query

'CHEMBL203'

In [6]:
#selecting the desired data set
activity = new_client.activity
erbb1_ic50 = activity.filter(target_chembl_id = selected_query).filter(standard_type = "IC50")
erbb1_df = pd.DataFrame(erbb1_ic50)

In [105]:
#removing any missing values in the "standard value" column
erbb1_df_naremove = erbb1_df[erbb1_df.standard_value.notna()]
erbb1_df_naremove.shape

(15336, 46)

In [108]:
#removing missing cananonical smiles
erbb1_df_naremove_smiles = erbb1_df_naremove[erbb1_df.canonical_smiles.notna()]
erbb1_df_naremove_smiles.shape

C:\Users\jhamb\AppData\Local\Temp\ipykernel_5000\1831612217.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  erbb1_df_naremove_smiles = erbb1_df_naremove[erbb1_df.canonical_smiles.notna()]


(15329, 46)

In [109]:
#Getting a list of columns from the erbb1_df
erbb1_df_naremove_smiles.columns

Index(['action_type', 'activity_comment', 'activity_id', 'activity_properties',
       'assay_chembl_id', 'assay_description', 'assay_type',
       'assay_variant_accession', 'assay_variant_mutation', 'bao_endpoint',
       'bao_format', 'bao_label', 'canonical_smiles', 'data_validity_comment',
       'data_validity_description', 'document_chembl_id', 'document_journal',
       'document_year', 'ligand_efficiency', 'molecule_chembl_id',
       'molecule_pref_name', 'parent_molecule_chembl_id', 'pchembl_value',
       'potential_duplicate', 'qudt_units', 'record_id', 'relation', 'src_id',
       'standard_flag', 'standard_relation', 'standard_text_value',
       'standard_type', 'standard_units', 'standard_upper_value',
       'standard_value', 'target_chembl_id', 'target_organism',
       'target_pref_name', 'target_tax_id', 'text_value', 'toid', 'type',
       'units', 'uo_units', 'upper_value', 'value'],
      dtype='object')

In [110]:
erbb1_df_naremove_smiles['data_validity_comment'].value_counts()

Outside typical range            314
Potential transcription error     53
Name: data_validity_comment, dtype: int64

In [111]:
#selecting desired columns
selected_columns = ['canonical_smiles', 'molecule_chembl_id','bao_label', 'standard_units', 'standard_value', 'pchembl_value', 'data_validity_comment']
erbb1_df_select_columns = erbb1_df_naremove_smiles[selected_columns]
erbb1_df_select_columns.head()


,canonical_smiles,molecule_chembl_id,bao_label,standard_units,standard_value,pchembl_value,data_validity_comment
0,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...,CHEMBL68920,single protein format,nM,41.0,7.39,None
1,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...,CHEMBL68920,cell-based format,nM,300.0,6.52,None
2,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...,CHEMBL68920,cell-based format,nM,7820.0,5.11,None
3,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,CHEMBL69960,single protein format,nM,170.0,6.77,None
4,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,CHEMBL69960,cell-based format,nM,40.0,7.40,None


In [112]:
erbb1_df_err = erbb1_df_select_columns[erbb1_df_select_columns['data_validity_comment'] == 'Outside typical range' ]
erbb1_df_err.head()

,canonical_smiles,molecule_chembl_id,bao_label,standard_units,standard_value,pchembl_value,data_validity_comment
7,CC(=C(C#N)C#N)c1ccc(NC(=O)CCC(=O)O)cc1,CHEMBL306988,single protein format,nM,500000.0,None,Outside typical range
8,O=C(O)/C=C/c1ccc(O)cc1,CHEMBL66879,single protein format,nM,3000000.0,None,Outside typical range
11,COc1cc(/C=C(\C#N)C(=O)O)cc(OC)c1O,CHEMBL76979,single protein format,nM,264000.0,None,Outside typical range
13,COc1cc(C=C(C#N)C#N)cc(OC)c1,CHEMBL77724,single protein format,nM,1250000.0,None,Outside typical range
14,N#CC(C#N)=Cc1ccc(N2CCOCC2)cc1,CHEMBL420385,single protein format,nM,625000.0,None,Outside typical range


In [113]:
erbb1_df_err.loc[:,'standard_value'].max()

'960063.59'

In [115]:
#filtering for data_validity_comment = None
erbb1_df_err_rm = erbb1_df_select_columns[erbb1_df_select_columns['data_validity_comment'].isnull()]
erbb1_df_err_rm.shape

(14962, 7)

In [116]:
#looking at the bao labels (experimental design) and looking at the number of observations for each bao label
erbb1_df_err_rm['bao_label'].unique()
erbb1_df_err_rm['bao_label'].value_counts()

single protein format    7131
cell-based format        4347
assay format             2866
protein format            537
cell membrane format       42
cell-free format           39
Name: bao_label, dtype: int64

In [ ]:
#convert 'standard_units' to a common unit? 

In [117]:
#look at the range for IC50 values - log transformation (pchembl)?
column_range = float(erbb1_df_err_rm.loc[:,'standard_value'].max()) - float(erbb1_df_err_rm.loc[:,'standard_value'].min())
column_range

99899.996

In [118]:
erbb1_df_err_rm.loc[:,'pchembl_value'] = pd.to_numeric(erbb1_df_err_rm.loc[:,'pchembl_value'])
chemblp_range = float(erbb1_df_err_rm.loc[:,'pchembl_value'].max()) - float(erbb1_df_err_rm.loc[:,'pchembl_value'].min())
chemblp_range

C:\Users\jhamb\AppData\Local\Temp\ipykernel_5000\900186712.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  erbb1_df_err_rm.loc[:,'pchembl_value'] = pd.to_numeric(erbb1_df_err_rm.loc[:,'pchembl_value'])
C:\Users\jhamb\AppData\Local\Temp\ipykernel_5000\900186712.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  erbb1_df_err_rm.loc[:,'pchembl_value'] = pd.to_numeric(erbb1_df_err_rm.loc[:,'pchembl_value'])


7.0

In [119]:
#Selecting 'single protein format' boa labels
erbb1_df_spf = erbb1_df_err_rm[erbb1_df_err_rm['bao_label'] == 'single protein format']
erbb1_df_spf.shape

(7131, 7)

In [120]:
#Removing duplicate compounds 
erbb1_spf_noduplicates_df = erbb1_spf_df.drop_duplicates(subset=['molecule_chembl_id'])
erbb1_spf_noduplicates_df.shape

(5385, 7)